# Zika Plasmablasts: tutorial
In this tutorial, we'll analyze sequencing data derived from single-cell PCR of plasmablasts from four subjects acutely infected with Zika virus (ZIKV). These data are published [here](https://immunology.sciencemag.org/content/2/14/eaan6809.long). In this notebook, we'll demonstrate some of the tools we use to analyze B cell receptore data and we'll generate several of the plots used in the paper.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from collections import Counter
import pickle

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.legend_handler import HandlerPatch

from abutils.core.lineage import group_lineages, donut
from abutils.utils.color import hex_to_rgb

%matplotlib inline

## Donors and colors

There are four donors, named (very logically): Donors 1-4. All donors, except Donor 1, were seropositive for Dengue virus (DENV) prior to ZIKV infection. The pre-existing anti-DENV antibdies were expected to cross-react substantially with ZIKV. 

Also of note, the colors used for the donors were drawn from one of my favorite color palette sites: [Wes Anderson Palettes]/(https://wesandersonpalettes.tumblr.com/). If you'd prefer a more tradition (primary) color palette, uncomment the third line of the code block below.

In [ ]:
donors = ['donor_1', 'donor_2', 'donor_3', 'donor_4']
colors = ['#004E64', '#5FAD56', '#3AAED8', '#F78154']
# colors = sns.color_palette(n_colors=4)
color_dict = {d: c for d, c in zip(donors, colors)}

def rgba_colors(alpha):
    clist_rgb = [[rgb / 256 for rgb in hex_to_rgb(c)] for c in colors]
    return [rgb + [alpha] for rgb in clist_rgb]

sns.palplot(colors)

## Load sequencing data
Slightly different from the previous notebook, this set sequencing data has been slightly pre-processed and saved to a single file using `pickle`. Sequences are already assigned to pairs, each pair has been populated with the `subject` attribute, and sequences have been assigned to clonal lineages. The first two lines open the binary `pickle` file and save the contents to a variable called `all_pairs`.

In [ ]:
with open('../data/zika_mAbs.pkl', 'rb') as f:
    pairs = pickle.load(f)

d1_pairs = [p for p in pairs if p.subject == 'donor_1']
d2_pairs = [p for p in pairs if p.subject == 'donor_2']
d3_pairs = [p for p in pairs if p.subject == 'donor_3']
d4_pairs = [p for p in pairs if p.subject == 'donor_4']

## Group sequences into lineages
Building on the `Sequence` and `Pair` objects that we learned about earlier, ab[x] also contains a `Lineage` object, designed to represent antibody clonal lineages. Using the built-in `group_lineages` helper function, we can group `Pair` objects into lineages.

In [ ]:
d1_lineages = group_lineages(d1_pairs)
d2_lineages = group_lineages(d2_pairs)
d3_lineages = group_lineages(d3_pairs)
d4_lineages = group_lineages(d4_pairs)

<br>Let's pick a lineage from Donor 3 so that we can learn a bit more about the `Lineage` object

In [ ]:
l = d3_lineages[6]

<br>Lineage objects have a `name` attribute as well as a `size` attribute

In [ ]:
print(l.name)

In [ ]:
print(l.size())

<br>If we pass the `pairs_only` keyword when checking the lineage size, it returns the number of paired heavy/light chains (excluding unpaired sequences)

In [ ]:
print(l.size(pairs_only=True))

<br>`Lineage` objects also have a built-in method for making dot alignments. This is slightly different from the dot alignment we made earlier, as calling the lineage-specific method aligns all sequences to the 'Unmutated Common Ancestor" (or UCA).

In [ ]:
print(l.dot_alignment(seq_field='vdj_aa', chain='heavy'))
print('\n')

## Lineage "donut" plots

"Donut" plots have long been used to visualize the distribution of lineages within a repertoire. Although pie-type plots have a number of deficiencies, we're not  trying to be precisely quantitative. The main purpose is to provide a easy way to visually compare the relative oligoclonality of one or more repertoires. In this case, it's obvious that the plasmablast repertoire of Donor 1 (the donor without existing serum reactivity to DENV) is less oligoclonal than the reperoires of Donors 2-4.

To make plots for other donors, replace `d1_lineages` with the variable corresponding to lineages from a different donor, and change the donor name in the second line (`"donor_1"` in the code block below).

In [ ]:
donut(d1_lineages,
      monochrome_color=cdict['donor_1'], 
      shuffle_colors=True,
      text_kws={'size': 40},
      seed=12,
#     figfile='./figures/donor1_donut.pdf'
     )

## Nucleotide mutation frequencies

In [ ]:
def get_nt_mutations(pairs, chain, just_pairs=True):
    if just_pairs:
        pairs = [p for p in pairs if p.is_pair]
    if chain == 'heavy':
        heavies = [p for p in pairs if p.heavy is not None]
        muts = [{'muts': s.heavy['var_muts_nt']['num'],
                 'donor': s.subject.replace('donor_', ''),
                 'color': color_dict[s.subject]} for s in heavies]
    else:
        lights = [p for p in pairs if p is not None]
        muts = [{'muts': s.light['var_muts_nt']['num'],
                 'donor': s.subject.replace('donor_', ''),
                 'color': color_dict[s.subject]} for s in lights]
    return pd.DataFrame(muts)


def mutation_violinplot(df, ylabel, figfile=None):
    sns.set_style('whitegrid')
    plt.figure(figsize=(5, 3.5))

    ax = sns.violinplot(x='donor', y='muts', palette=colors, data=mut_df,
                        cut=0, linewidth=1.5, saturation=1)

    # make the plot borders visible and black
    for position in ['right', 'left', 'top', 'bottom']:
            ax.spines[position].set_visible(True)
            ax.spines[position].set_color('k')

    # set axis limits
    ax.set_ylim([0, 50])

    # adjust tick and axis labels
    ax.tick_params(axis='y', labelsize=12)
    ax.tick_params(axis='x', labelsize=14)
    plt.ylabel(ylabel, size=14)
    plt.xlabel('Donor', size=14)
    
    if figfile is None:
        plt.show()
    else:
        plt.tight_layout()
        plt.savefig(figfile)

In [ ]:
mut_df = get_nt_mutations(pairs, 'heavy')
mutation_violinplot(mut_df,
                    'Heavy chain NT mutations',
#                     figfile='./figures/ZIKV_nt-mutations_heavy.pdf'
                   )

In [ ]:
mut_df = get_nt_mutations(pairs, 'light')
mutation_violinplot(mut_df,
                    'Light chain NT mutations',
#                     figfile='./figures/ZIKV_nt-mutations_light.pdf'
                   )

## Mutation bubbleplots

In [ ]:
def mutation_bubbleplot(lineages, figfile=None):
    # read mutation data
    xs = []
    ys = []
    cs = []
    areas = []
    for l in lineages:
        if len(l.heavies) == 1:
            continue
        xs.append(min([100. - p.heavy['nt_identity']['v'] for p in l.heavies]))
        ys.append(max([100. - p.heavy['nt_identity']['v'] for p in l.heavies]))
        cs.append(color_dict[l.heavies[0].subject])
        areas.append(45 * len(l.heavies))
    
    sns.set_style('white')
    plt.figure(figsize=(6, 6))
    
    # plot the diagonal reference line
    plt.plot((0, 16), (0, 16), '--', c='k', linewidth=1, alpha=0.7)
    
    # plot the bubbles
    outline = plt.scatter(xs, ys, c='w', s=areas, linewidths=1, edgecolor=cs)
    outline.set_alpha(0.75)
    fill = plt.scatter(xs, ys, c=cs, s=areas)
    fill.set_alpha(0.3)

    # style the plot
    ax = plt.gca()
    ax.set_ylim(0, 16)
    ax.set_xlim(0, 16)
    ax.tick_params(axis='y', labelsize=12)
    ax.tick_params(axis='x', labelsize=12)
    plt.ylabel('Maximum HC mutation (% nucleotide)', fontsize=14)
    plt.xlabel('Minimum HC mutation (% nucleotide)', fontsize=14)

    # build the legend
    class HandlerEllipse(HandlerPatch):

        def create_artists(self, legend, orig_handle,
                           xdescent, ydescent, width, height, fontsize, trans):
            center = 0.5 * width - 0.5 * xdescent, 0.5 * height - 0.5 * ydescent
            p = mpl.patches.Circle(xy=center)
            self.update_prop(p, orig_handle, legend)
            p.set_transform(trans)
            return [p]
    patches = []
    for d, c, rgba in zip(donors, rgba_colors(0.75), rgba_colors(0.35)):
        d = d.replace('donor_', 'Donor ')
        patch = mpl.patches.Circle((-0.5, -0.5), 0.1,
                                   facecolor=rgba,
                                   edgecolor=c,
                                   linewidth=1)
        plt.gca().add_patch(patch)
        patches.append(patch)
    plt.legend(patches, [d.replace('donor_', 'Donor ') for d in donors],
               handler_map={mpl.patches.Circle: HandlerEllipse()},
               fontsize=12,
               frameon=False,
               loc='lower right')
    
    # save or show
    if figfile is None:
        plt.show()
    else:
        plt.tight_layout()
        plt.savefig(figfile)

In [ ]:
all_lineages = d1_lineages + d2_lineages + d3_lineages + d4_lineages
mutation_bubbleplot(all_lineages,
#                     figfile='./figures/mutation_bubbleplot.pdf'
                   )